# Initialisation

Import libraries, load setting and load data.

In [2]:
import numpy as np
import utils
import json

path_base = "../"

# Load settings
with open(path_base+"SETTINGS.json", 'r') as fid:
    settings = json.load(fid)
    print("Settings loaded.")
    
# Load store info
store_info = utils.loadStoreInfo(path_base+settings["STORE_INFO_PATH"])
print("Store information: {} lines loaded".format(len(store_info)))

# Load training data
(input_data, output_sales, output_customers) = utils.loadTrainingData(path_base+settings["TRAIN_DATA_PATH"])
print("Training data: {} lines loaded".format(len(input_data)))

Settings loaded.
Store information: 1115 lines loaded
Training data: 1017209 lines loaded
